In [2]:
import pandas as pd
import json
import numpy as np
from surprise import Dataset, Reader

In [3]:
checkin_path="/home/jaume/Escritorio/Ultim_Semestre/Comp.ML/Practicas/project/Yelp-JSON/Yelp JSON/yelp_dataset/yelp_academic_dataset_checkin.json"
business_path="/home/jaume/Escritorio/Ultim_Semestre/Comp.ML/Practicas/project/Yelp-JSON/Yelp JSON/yelp_dataset/yelp_academic_dataset_business.json"
review_path="/home/jaume/Escritorio/Ultim_Semestre/Comp.ML/Practicas/project/Yelp-JSON/Yelp JSON/yelp_dataset/yelp_academic_dataset_review.json"
tip_path="/home/jaume/Escritorio/Ultim_Semestre/Comp.ML/Practicas/project/Yelp-JSON/Yelp JSON/yelp_dataset/yelp_academic_dataset_tip.json"
user_path="/home/jaume/Escritorio/Ultim_Semestre/Comp.ML/Practicas/project/Yelp-JSON/Yelp JSON/yelp_dataset/yelp_academic_dataset_user.json"

In [4]:
with open(business_path, "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

business = pd.DataFrame(data)
with open(review_path, "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

review = pd.DataFrame(data)

In [5]:
business_filtered = business[business["city"] == "Sparks"]
business_ids = business_filtered["business_id"]

filtered_reviews= review[review["business_id"].isin(business_ids)]


In [10]:
# Definir umbrales
MIN_USER_REV = 2  # Mínimo de reviews por usuario
MIN_BUSS_REV = 10  # Mínimo de reviews por negocio

# Contar reviews por usuario y negocio
user_counts = filtered_reviews['user_id'].value_counts()
business_counts = filtered_reviews['business_id'].value_counts()

# Filtrar usuarios y negocios con suficientes reviews
filtered_reviews = filtered_reviews[
    (filtered_reviews['user_id'].isin(user_counts[user_counts >= MIN_USER_REV].index)) &
    (filtered_reviews['business_id'].isin(business_counts[business_counts >= MIN_BUSS_REV].index))
]

# Identificar duplicados antes de pivotear
duplicates = filtered_reviews.duplicated(subset=['user_id', 'business_id'], keep=False)

if duplicates.any():
    print("¡Advertencia! Hay usuarios con múltiples reviews para el mismo negocio.")
    print(filtered_reviews[duplicates])


            

¡Advertencia! Hay usuarios con múltiples reviews para el mismo negocio.
                      review_id                 user_id  \
281      eBrVkzHhHKZxyVS1SPKbJw  1PJ-RjMqXHeymSkLRzuIdA   
1384     GgK9ub1jfTwv-RGfWvOJ7A  vwOX46OZGisDjSWIJOsWsg   
6566     y4njqBzKkq3QSFVTZQOCfw  7DurpyzYkWZ-GfDh8-B-GA   
7758     uw9ooDApUnbrVmhaRwyijQ  A4YsDUocwpqCjtaMS32R6g   
9210     wygIvfvRq6rcwDAkkEe_HA  P7U7oaOTEGJlDJOYSC6VMg   
...                         ...                     ...   
6983611  Sic1ET5U5o-JOgdko3_01w  OWpS0xRmk6FxDyHV0vxQmQ   
6984174  fdIElkNDZyfwWEinxRmUZw  eGZvv1U1tRoV2UJqv4AIhg   
6984274  1jDy-jiNzh0l2ZM4Tgm0yw  R9IJ1Byr27n6wXj3QvJ2Lg   
6988042  Qs0N7_eTxh-5JesxPsiekA  zv7tpu7xeaNyAeFG03d2CA   
6988497  IWbOZVvrcK5VSXKBGkn_jg  iDKLlYHBmb-Dr1CUuvoJTQ   

                    business_id  stars  useful  funny  cool  \
281      nOyphAl0JQ8JrvNi93pclQ    2.0       3      1     1   
1384     xR1Wkmrm3yoAJuxPmS0vUw    1.0       0      0     1   
6566     JYDd5sgn18DqDnTnq_tBR

In [11]:
from surprise import Dataset, Reader

# Mantener solo las últimas reviews antes de cargar en Surprise
filtered_reviews = (filtered_reviews.sort_values('date')
                    .drop_duplicates(subset=['user_id', 'business_id'], keep='last'))

# Definir el formato de los datos para Surprise
reader = Reader(rating_scale=(1, 5))  # Ajusta según tu escala de calificación
data = Dataset.load_from_df(filtered_reviews[['user_id', 'business_id', 'stars']], reader)

# Construir conjunto de entrenamiento
trainset = data.build_full_trainset()


In [12]:
import pandas as pd

# Crear una matriz vacía
user_ids = list(trainset.all_users())  # Lista de IDs de usuarios en Surprise
business_ids = list(trainset.all_items())  # Lista de IDs de negocios en Surprise
ratings_matrix = pd.DataFrame(index=user_ids, columns=business_ids)

# Llenar la matriz con las calificaciones disponibles
for u, i, r in trainset.all_ratings():
    ratings_matrix.at[u, i] = r

# Imprimir la matriz resultante
print(ratings_matrix)


      0    1    2    3    4    5    6    7    8    9    ...  876  877  878  \
0     3.0  4.0  3.0  3.0  3.0  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
1     NaN  NaN  NaN  NaN  NaN  5.0  3.0  5.0  2.0  NaN  ...  NaN  NaN  NaN   
2     NaN  NaN  4.0  NaN  NaN  5.0  4.0  5.0  NaN  NaN  ...  NaN  NaN  NaN   
3     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  4.0  3.0  ...  NaN  NaN  NaN   
4     NaN  NaN  5.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
9825  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
9826  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
9827  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
9828  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
9829  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   

      879  880  881  882  883  884  885  
0     NaN  NaN  NaN  

In [13]:
ratings_matrix

,0,1,2,3,4,5,6,7,8,9,...,876,877,878,879,880,881,882,883,884,885
0,3.0,4.0,3.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,5.0,3.0,5.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,4.0,NaN,NaN,5.0,4.0,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9825,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9827,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9828,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### USER-BASED RECOMENDEDATIONS

In [14]:
ratings_user_filled = ratings_matrix.apply(lambda row: row.fillna(row.mean()), axis=1)


/tmp/ipykernel_1272787/1699230609.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ratings_user_filled = ratings_matrix.apply(lambda row: row.fillna(row.mean()), axis=1)


In [15]:
ratings_user_filled 

,0,1,2,3,4,5,6,7,8,9,...,876,877,878,879,880,881,882,883,884,885
0,3.000000,4.000000,3.000000,3.000000,3.000000,3.555556,3.555556,3.555556,3.555556,3.555556,...,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556
1,4.000000,4.000000,4.000000,4.000000,4.000000,5.000000,3.000000,5.000000,2.000000,4.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
2,4.285714,4.285714,4.000000,4.285714,4.285714,5.000000,4.000000,5.000000,4.285714,4.285714,...,4.285714,4.285714,4.285714,4.285714,4.285714,4.285714,4.285714,4.285714,4.285714,4.285714
3,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,4.000000,3.000000,...,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333
4,4.500000,4.500000,5.000000,4.500000,4.500000,4.500000,4.500000,4.500000,4.500000,4.500000,...,4.500000,4.500000,4.500000,4.500000,4.500000,4.500000,4.500000,4.500000,4.500000,4.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9825,4.250000,4.250000,4.250000,4.250000,4.250000,4.250000,4.250000,4.250000,4.250000,4.250000,...,4.250000,4.250000,4.250000,4.250000,4.250000,4.250000,4.250000,4.250000,4.250000,4.250000
9826,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
9827,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
9828,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### BUSINESS-BASED RECOMENDEDATIONS

In [16]:
ratings_business_filled = ratings_matrix.apply(lambda col: col.fillna(col.mean()), axis=0)


/tmp/ipykernel_1272787/3369165125.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ratings_business_filled = ratings_matrix.apply(lambda col: col.fillna(col.mean()), axis=0)


In [17]:
ratings_business_filled 

,0,1,2,3,4,5,6,7,8,9,...,876,877,878,879,880,881,882,883,884,885
0,3.000000,4.0,3.000000,3.000000,3.000000,4.348993,4.284974,4.093023,2.633776,2.588785,...,3.8125,4.636364,4.409091,4.282051,3.076923,3.06383,3.526316,4.441176,3.903614,3.975
1,3.727273,2.0,4.073529,3.616667,3.929293,5.000000,3.000000,5.000000,2.000000,2.588785,...,3.8125,4.636364,4.409091,4.282051,3.076923,3.06383,3.526316,4.441176,3.903614,3.975
2,3.727273,2.0,4.000000,3.616667,3.929293,5.000000,4.000000,5.000000,2.633776,2.588785,...,3.8125,4.636364,4.409091,4.282051,3.076923,3.06383,3.526316,4.441176,3.903614,3.975
3,3.727273,2.0,4.073529,3.616667,3.929293,4.348993,4.284974,4.093023,4.000000,3.000000,...,3.8125,4.636364,4.409091,4.282051,3.076923,3.06383,3.526316,4.441176,3.903614,3.975
4,3.727273,2.0,5.000000,3.616667,3.929293,4.348993,4.284974,4.093023,2.633776,2.588785,...,3.8125,4.636364,4.409091,4.282051,3.076923,3.06383,3.526316,4.441176,3.903614,3.975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9825,3.727273,2.0,4.073529,3.616667,3.929293,4.348993,4.284974,4.093023,2.633776,2.588785,...,3.8125,4.636364,4.409091,4.282051,3.076923,3.06383,3.526316,4.441176,3.903614,3.975
9826,3.727273,2.0,4.073529,3.616667,3.929293,4.348993,4.284974,4.093023,2.633776,2.588785,...,3.8125,4.636364,4.409091,4.282051,3.076923,3.06383,3.526316,4.441176,3.903614,3.975
9827,3.727273,2.0,4.073529,3.616667,3.929293,4.348993,4.284974,4.093023,2.633776,2.588785,...,3.8125,4.636364,4.409091,4.282051,3.076923,3.06383,3.526316,4.441176,3.903614,3.975
9828,3.727273,2.0,4.073529,3.616667,3.929293,4.348993,4.284974,4.093023,2.633776,2.588785,...,3.8125,4.636364,4.409091,4.282051,3.076923,3.06383,3.526316,4.441176,3.903614,3.975
